<a href="https://colab.research.google.com/github/bilik49/statistical_analysis/blob/main/analysis%20of%20categorical%20features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [114]:
import numpy as np
import pandas as pd
import scipy.stats as st

In [115]:
df = pd.read_csv('/content/drive/MyDrive/data_big.csv')

In [116]:
df.head()

,Unnamed: 0,X.1,depressed.mood.1,anxiety.1,suspiciousness.1,irritability.1,craving.to.alcohol.1,weakness.1,insomia.1,headache.1,...,combined2.9,HR.9,SBP.9,DBP.9,MBP.9,SV.9,CO.9,SI.9,CI.9,TPR.9
0,1,1,1,1,0,1,1,1,1,0,...,0,68.0,108.0,70.0,83.0,74.0,5.0,40.0,2.7,1321.0
1,2,2,1,1,0,0,1,1,2,1,...,9,63.0,114.0,70.0,85.0,123.0,7.7,60.0,3.8,879.0
2,3,3,1,1,0,0,0,2,1,0,...,0,64.0,120.0,80.0,93.0,106.0,6.8,55.0,3.5,1098.0
3,4,4,2,2,0,0,0,2,0,0,...,0,56.0,124.0,90.0,101.0,90.0,5.1,43.0,2.4,1600.0
4,5,5,1,1,0,0,2,2,1,0,...,0,66.0,116.0,78.0,90.0,90.0,5.9,45.0,2.2,1228.0


Взглянем на количество уникальных значений в каждой колонке.

In [117]:
df.nunique().head(10)

Unnamed: 0              34
X.1                     34
depressed.mood.1         2
anxiety.1                3
suspiciousness.1         2
irritability.1           3
craving.to.alcohol.1     3
weakness.1               3
insomia.1                3
headache.1               3
dtype: int64

### 1. Энтропия и коэффициенты неопределенности.
Возьмем два признака, например, бессоница и слабость.  
Посчитаем таблицу частот.

In [118]:
pd.crosstab(index=df["insomia.1"],columns=df["weakness.1"], margins=True)

weakness.1,0,1,2,All
insomia.1,,,,
0,0,2,5,7
1,1,5,4,10
2,2,8,7,17
All,3,15,16,34


Посчитаем ту же таблицу, но уже с вероятностями.

In [119]:
ct = pd.crosstab(index=df["insomia.1"],columns=df["weakness.1"], margins=True, normalize=True)
ct

weakness.1,0,1,2,All
insomia.1,,,,
0,0.000000,0.058824,0.147059,0.205882
1,0.029412,0.147059,0.117647,0.294118
2,0.058824,0.235294,0.205882,0.500000
All,0.088235,0.441176,0.470588,1.000000


Вычислим энтропию и коэф-ты неопределенности.

In [120]:
H_Y = st.entropy(ct.iloc[-1,:-1])
H_X = st.entropy(ct.iloc[:-1,-1])
H_XY = st.entropy(ct.iloc[:-1,:-1].stack())
I_XY = H_X + H_Y - H_XY
J_XY = I_XY / H_Y 
J_YX = I_XY / H_X 
J = 2 * I_XY / (H_X + H_Y)
entropy_results = pd.DataFrame([{'H_X':H_X,
                                'H_Y':H_Y,
                                'H_XY':H_XY,
                                'I_XY':I_XY,
                                'J_XY':J_XY,
                                'J_YX':J_YX,
                                'J':J}])
entropy_results

,H_X,H_Y,H_XY,I_XY,J_XY,J_YX,J
0,1.031894,0.929948,1.918447,0.043396,0.046664,0.042054,0.04424


### 2. Критерий Фишера.  
Взглянем на колонки, содержащие только две категории.

In [121]:
df.columns[df.nunique()==2]

Index(['depressed.mood.1', 'suspiciousness.1', 'polyuria.1', 'diarrhea.1',
       'anxiety.2', 'irritability.2', 'polyuria.2',
       'transient.hallusinations.2', 'vomiting.2', 'diarrhea.2',
       'chest.pain.2', 'hyperemia.2', 'depressed.mood.3', 'anxiety.3',
       'irritability.3', 'craving.to.alcohol.3', 'headache.3', 'polyuria.3',
       'transient.hallusinations.3', 'vomiting.3', 'anoreksia.3', 'diarrhea.3',
       'chest.pain.3', 'hyperemia.3', 'depressed.mood.9', 'anxiety.9',
       'weakness.9', 'insomnia.9', 'headache.9', 'tremor.9', 'polyuria.9',
       'sweating.9', 'anoreksia.9', 'thirst.9', 'hyperemia.9'],
      dtype='object')

Посмотрим на связь депрессивного настроения и тревожности на 9 день.  
Для этого проверим гипотезу о независимости двух переменных с помощью точного критерия Фишера на 5% уровне значимости.  
Сначала составим таблицу частот.

In [122]:
ct = pd.crosstab(index=df["depressed.mood.9"],columns=df["anxiety.9"], margins=True)
ct

anxiety.9,0.0,1.0,All
depressed.mood.9,,,
0.0,29,1,30
1.0,1,2,3
All,30,3,33


Вычислим pvalue для данной таблицы.

In [123]:
_, pvalue = st.fisher_exact(ct.iloc[:-1,:-1])
pvalue

0.016678885630498526

Данное значение pvalue < 0.05, следовательно, отвергаем нулевую гипотезу о независимости признаков.  
Таким образом, утверждение, что на 9 день после употребления алкоголя люди, не страдающие депрессией, не склонны к тревожности, статистически значимо. 

### 3. Критерий Кульбака.  
Проверим ту же гипотезу о независимости двух признаков.

In [124]:
def Kullback(ct):
  L = 0
  for i in range(ct.shape[0]-1):
    for j in range(ct.shape[1]-1):
      L += ct.iloc[i,j]*np.log(ct.iloc[i,j] * ct.iloc[-1,-1] / (ct.iloc[-1,j]*ct.iloc[i,-1]))
  L *= 2
  pvalue = 1 - st.chi2.cdf(L,1)
  return L, pvalue

In [125]:
ct

anxiety.9,0.0,1.0,All
depressed.mood.9,,,
0.0,29,1,30
1.0,1,2,3
All,30,3,33


In [126]:
_, pvalue = Kullback(ct)
pvalue

0.006107825360364028

Данное значение pvalue < 0.05, следовательно, отвергаем нулевую гипотезу о независимости признаков.

### 4. Критерий Мак-Немара.  
Рассмотрим уровень настроения в 1 день и 3 день.

In [127]:
ct = pd.crosstab(index=df["depressed.mood.1"], columns=df["depressed.mood.3"], margins=True)
ct

depressed.mood.3,0.0,1.0,All
depressed.mood.1,,,
1,7,11,18
2,1,4,5
All,8,15,23


In [128]:
from statsmodels.stats.contingency_tables import mcnemar
pvalue = mcnemar(ct.iloc[:-1,:-1], exact=False).pvalue
pvalue

0.02334220201289086

Данное значение pvalue < 0.05, следовательно, отвергаем нулевую гипотезу о неизменности распределения.  
Таким образом, изменение уровня настроения у людей в 1 и 3 дни после употребления алкоголя статистически значим.

### 5. Критерий Кохрена.  
Рассмотрим степень слабости в течение 3 дней после употребления.

In [129]:
ct = pd.concat([df["polyuria.1"],df["polyuria.2"],df["polyuria.9"].astype('Int64')], axis=1)
ct = ct.dropna()
ct["Sum"] = ct.sum(axis=1).astype(int)
ct = ct.append(pd.DataFrame(ct.sum(axis=0), columns=["Sum"]).T)
ct

,polyuria.1,polyuria.2,polyuria.9,Sum
0,0,0,0,0
1,0,0,1,1
2,0,0,0,0
3,0,0,0,0
4,0,0,0,0
5,0,0,0,0
6,0,0,0,0
7,1,0,0,1
8,0,0,0,0
9,0,0,0,0


In [130]:
from statsmodels.stats.contingency_tables import cochrans_q
pvalue = cochrans_q(ct.iloc[:-1,:-1]).pvalue
pvalue

0.04393693362340742

Данное значение pvalue < 0.05, следовательно, отвергаем нулевую гипотезу о неизменности распределения во все моменты времени.  
Таким образом, изменение распределения в течение проведения исследования статистически значимо.